Collect EIA data on generation from nonfossil fuel sources

In [ ]:
import json
import requests

In [ ]:
all_data = pd.DataFrame({'CAL_COL':np.zeros((8760,))})
for fuel in ['COL', 'WAT', 'NG', 'NUC', 'OTH', 'OIL', 'WND', 'SUN']:
    for region in ['CAL', 'SW', 'NW']:
        url = 'http://api.eia.gov/series/?api_key=f9839ba3cae2a9f5fae81a5ebe2eb98c&series_id=EBA.'+region+'-ALL.NG.'+fuel+'.HL'
        r = requests.get(url)
        j = r.json()
        data_here = pd.DataFrame(j['series'][0]['data'], columns=['Timestamp', 'MWh'])
        data_here['dt'] = pd.to_datetime(data_here['Timestamp'])
        data_here = data_here[::-1].reset_index(drop = True).copy(deep=True)
        dt_here = data_here['dt'].astype(str).str.split(' ', expand=True)
        inds = np.where(pd.to_datetime(dt_here[0]).dt.year == 2019)[0]
        all_data[region+'_'+fuel] = data_here.loc[inds]['MWh'].values


In [ ]:
for fuel in ['COL', 'WAT', 'NG', 'NUC', 'OTH', 'OIL', 'WND', 'SUN']:
    all_data['WECC_'+fuel] = all_data['SW_'+fuel] + all_data['NW_'+fuel] + all_data['CAL_'+fuel]
all_data['WECC_combustion'] = all_data.loc[:, ['WECC_COL', 'WECC_NG', 'WECC_OIL', 'WECC_OTH']].sum(axis=1)
all_data['WECC_notcombustion'] = all_data.loc[:, ['WECC_WAT', 'WECC_NUC', 'WECC_WND', 'WECC_SUN']].sum(axis=1)

In [ ]:
all_data.to_csv('region_eia_generation_data_2019.csv')
